<a href="https://colab.research.google.com/github/hepham/graduation_thesis/blob/main/XGBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from functools import reduce
import requests
from sklearn.preprocessing import StandardScaler
from scipy import optimize
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path_train='/content/gdrive/MyDrive/datakltn/datatest.csv'
path_test='/content/gdrive/MyDrive/datakltn/area0/test.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data_train=pd.read_csv(path_train)
data_test=pd.read_csv(path_test)
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48600 entries, 0 to 48599
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Position Label  48600 non-null  int64
 1   Beacon1         48600 non-null  int64
 2   RSSI1           48600 non-null  int64
 3   Beacon2         48600 non-null  int64
 4   RSSI2           48600 non-null  int64
 5   Beacon3         48600 non-null  int64
 6   RSSI3           48600 non-null  int64
 7   Beacon4         48600 non-null  int64
 8   RSSI4           48600 non-null  int64
 9   Beacon5         48600 non-null  int64
 10  RSSI5           48600 non-null  int64
 11  Beacon6         48600 non-null  int64
 12  RSSI6           48600 non-null  int64
 13  Beacon7         48600 non-null  int64
 14  RSSI7           48600 non-null  int64
 15  Beacon8         48600 non-null  int64
 16  RSSI8           48600 non-null  int64
 17  Beacon9         48600 non-null  int64
 18  RSSI9           48600 non-

In [ ]:
data_train.head()
print("shape:",data_train.shape)
data_train=data_train.abs()
data_test=data_test.abs()

shape: (48600, 21)


In [ ]:
data_test.head()

,Position Label,Beacon 1,RSS 1,Beacon 2,RSS 2,Beacon 3,RSS 3,Beacon 4,RSS 4,Beacon 5,RSS 5,Beacon 6,RSS 6,Beacon 7,RSS 7,Beacon 8,RSS 8
0,36,0.22,0.696,0.12,0.577,0.6,0.752,0.2,0.730,0.209,0.665,0.14,0.631,0.15,0.644,0.1844,0.556
1,38,0.22,0.701,0.12,0.585,0.6,0.744,0.2,0.711,0.209,0.663,0.14,0.622,0.15,0.656,0.1844,0.557
2,39,0.22,0.708,0.12,0.607,0.6,0.759,0.2,0.716,0.209,0.663,0.14,0.631,0.15,0.653,0.1844,0.574
3,33,0.22,0.691,0.12,0.566,0.6,0.760,0.2,0.711,0.209,0.673,0.14,0.642,0.15,0.645,0.1844,0.567
4,17,0.22,0.671,0.12,0.624,0.6,0.772,0.2,0.751,0.209,0.695,0.14,0.657,0.15,0.618,0.1844,0.609


In [ ]:
data_test.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Position Label,199.0,19.170854,1.151376e+01,0.0000,9.0000,19.0000,28.5000,39.0000
Beacon 1,199.0,0.220000,0.000000e+00,0.2200,0.2200,0.2200,0.2200,0.2200
RSS 1,199.0,0.672050,3.619793e-02,0.2780,0.6620,0.6750,0.6910,0.7090
Beacon 2,199.0,0.120000,1.391279e-17,0.1200,0.1200,0.1200,0.1200,0.1200
RSS 2,199.0,0.615020,4.747290e-02,0.2740,0.5960,0.6280,0.6470,0.6660
Beacon 3,199.0,0.600000,1.113023e-16,0.6000,0.6000,0.6000,0.6000,0.6000
RSS 3,199.0,0.763990,3.388230e-02,0.3480,0.7565,0.7660,0.7790,0.7960
Beacon 4,199.0,0.200000,2.782558e-17,0.2000,0.2000,0.2000,0.2000,0.2000
RSS 4,199.0,0.748603,3.775089e-02,0.3160,0.7440,0.7540,0.7630,0.8050
Beacon 5,199.0,0.209000,5.565115e-17,0.2090,0.2090,0.2090,0.2090,0.2090


In [ ]:
import sklearn
from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from tqdm.notebook import tqdm_notebook as tqdm
import xgboost as xgb
def prepare_data():
 # Selecting the Feature, by remove the unused feature 
  feature = ['Position Label']
  train_feature =data_train.drop(feature, axis=1)

  # Set The Target
  train_target = data_train["Position Label"]
  # Split Data
  from sklearn.model_selection import train_test_split, cross_val_score
  X_train, X_test, y_train, y_test = train_test_split(train_feature ,train_target, test_size=0.3, random_state=42)
  X_train=X_train.to_numpy()
  # for i in X_train:
  #   for j in i:
  #     if j<0:
  #       j=-j
  #       print(j)

  X_test=X_test.to_numpy()
  y_train=y_train.to_numpy()
  print(y_train[0])
  y_test = y_test .to_numpy()
  # transform = preprocessing.MinMaxScaler()
  # X_train = transform.fit_transform(X_train)
  # X_test = transform.transform(X_test)
  return X_train, X_test, y_train, y_test

# def prepare_trainer(model):
#   # sgd = SGDOptimizer(model, learning_rate=0.1, regularization=0.03)
#   sgd = AdaGradOptimizer(mo del, learning_rate=0.05, regularization=0.005)
#   loss_func = CrossEntropyLoss()
#   return sgd, loss_func

# def prepare_data_loader(X, y, batch_size):
#   n = X.shape[0]
#   permutation = np.random.permutation(n)
#   for i in range(0, n, batch_size):
#     j = i+batch_size if i+batch_size <= n else n
#     batch_x = X[permutation[i:j]]
#     # print("bath_x: ",batch_x)
#     batch_y = y[permutation[i:j]]
#     # print("batch y:",batch_y)
#     yield batch_x, batch_y

# def get_model(n_in, n_out):
#   np.random.seed(101)
#   model = MLP(n_in=n_in, hiddens=[256,40], activation="sigmoid")
#   # model=FC(n_in=16,n_out=40,activation=None)
#   return model

# class Config:
#   n_epoch = 100000
#   batch_size =128

X, Xtest, y, ytest = prepare_data()
model_xgb = xgb.XGBClassifier(random_state=100, n_estimators = 500)
model_xgb.fit(X, y)
acc = model_xgb.score(Xtest, ytest)
print(acc*100)

17
94.65706447187928


In [ ]:
train=xgb.DMatrix(X,y)
test=xgb.DMatrix(Xtest,ytest)
params = {
    "learning_rate": 0.01,
    "max_depth": 3
}
# training, we set the early stopping rounds parameter
model_xgb = xgb.train(params, 
          train, evals=[(train, "train"), (test, "validation")], 
          num_boost_round=10000, early_stopping_rounds=20)

[0]	train-rmse:21.9585	validation-rmse:22.416
Multiple eval metrics have been passed: 'validation-rmse' will be used for early stopping.

Will train until validation-rmse hasn't improved in 20 rounds.
[1]	train-rmse:21.7425	validation-rmse:22.1989
[2]	train-rmse:21.5287	validation-rmse:21.9832
[3]	train-rmse:21.3169	validation-rmse:21.7703
[4]	train-rmse:21.1073	validation-rmse:21.5587
[5]	train-rmse:20.8998	validation-rmse:21.3495
[6]	train-rmse:20.6943	validation-rmse:21.1428
[7]	train-rmse:20.4908	validation-rmse:20.9371
[8]	train-rmse:20.2894	validation-rmse:20.7327
[9]	train-rmse:20.0899	validation-rmse:20.531
[10]	train-rmse:19.8925	validation-rmse:20.3305
[11]	train-rmse:19.697	validation-rmse:20.1329
[12]	train-rmse:19.5035	validation-rmse:19.9363
[13]	train-rmse:19.3119	validation-rmse:19.7426
[14]	train-rmse:19.1222	validation-rmse:19.5499
[15]	train-rmse:18.9345	validation-rmse:19.361
[16]	train-rmse:18.7485	validation-rmse:19.173
[17]	train-rmse:18.5645	validation-rmse:18.9

In [ ]:
model_xgb.best_ntree_limit

3599

In [ ]:
model_xgb.predict(test)

array([ 2.03009567e+01,  3.74588318e+01,  2.51014576e+01,  2.37560616e+01,
        3.59735985e+01,  2.59625053e+01,  1.07852516e+01,  5.12899923e+00,
        2.19436188e+01,  2.70766659e+01,  1.46140337e+01,  1.48186773e-01,
        2.02183952e+01,  1.69503899e+01,  4.75223398e+00,  3.38426514e+01,
        1.55908623e+01,  3.59779663e+01,  7.08385086e+00,  3.11559849e+01,
        2.02989120e+01,  2.65956154e+01,  1.31632099e+01,  8.22175503e+00,
        5.45281696e+00,  2.67563667e+01,  3.60370216e+01,  2.02732296e+01,
        2.35654335e+01,  2.01087437e+01,  4.02048874e+00,  9.83606911e+00,
        1.14650936e+01,  3.13523221e+00,  2.22803555e+01,  1.81728382e+01,
        2.70733471e+01,  8.21076965e+00,  3.89855576e+01,  8.98229027e+00,
        3.11421776e+01,  2.32506027e+01,  3.82302094e+01,  5.80496597e+00,
        3.56371803e+01,  6.72356009e-01,  2.80806503e+01,  2.30019569e+01,
        1.40250826e+01,  3.44354248e+01,  6.00378036e+00,  2.04679251e+00,
        2.82464199e+01,  